In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-pred/train.csv


# House Price Prediction Project
## 1. Problem
**Goal**: Công việc của bạn là dự đoán giá bán của từng ngôi nhà. Đối với mỗi Id trong bộ kiểm tra, bạn phải dự đoán giá trị của biến SalePrice.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('/kaggle/input/house-price-pred/train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data = pd.read_csv('/kaggle/input/house-price-pred/train.csv', index_col = 'Id')
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

## 2. Feature Selection
-Chọn đặc trưng (cột) giúp cho model dự đoán chuẩn nhất.</br>
-Cần sử dụng `Feature Engineering` để tìm ra những feature cần cho build model ML

In [6]:
feature = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

## 3. Spliting The Dataset
#### 3.1. dataset -> X,y
* `Data`: dataset
* `X` : data[feature]
* `y` : target variable - ở đây là: `SalePrice`

In [7]:
X = data[feature]
y = data['SalePrice']

In [8]:
X.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9


In [9]:
y.head()

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64

#### 3.2 X,y -> X_train,y_train, X_valid,y_valid

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=0)

![markdown](https://i.meeymedia.com/mXN3G4Gu5GTIoIYuCrQ_aZ5PUu-orL193vPX9wIJ1Gk/rs:auto:803:0/g:no/czM6Ly9tZWV5bGFuZC1jbXMvdXBsb2Fkcy9pbWFnZXMvMjAyMi8xMi8wOC9zcGxpdC0xLTE2NzA0NzA1MTQucG5n.png)

![markdown](https://i.meeymedia.com/sYNTnSp9OKBa00lWKukHF8GbH2GqahYo06p6I7x3nRQ/rs:auto:803:0/g:no/czM6Ly9tZWV5bGFuZC1jbXMvdXBsb2Fkcy9pbWFnZXMvMjAyMi8xMi8wOC9zcGxpdC0yLTE2NzA0NzA1MTQucG5n.png)

**Tập huấn luyện**: Training set là tập dữ liệu được sử dụng để huấn luyện, giúp mô hình tìm hiểu các tính năng/mẫu ẩn trong dữ liệu.</br>
**Tập tối ưu**: Validation Set, đôi khi được gọi là development set hay dev set, là tập dữ liệu tách biệt với tập huấn luyện, được sử dụng để xác thực hiệu suất mô hình trong quá trình đào tạo.</br>
**Tập thử nghiệm**: Test set là một tập dữ liệu riêng biệt được sử dụng để kiểm tra mô hình sau khi hoàn thành quá trình huấn luyện.</br>


In [11]:
X_train.shape

(1168, 7)

In [12]:
X_valid.shape

(292, 7)

## 4. Training MachineLearning Model

In [13]:
dt_model = DecisionTreeRegressor(random_state = 1)

In [14]:
# đưa data vào model
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [15]:
y_pred = dt_model.predict(X_valid.head())

In [16]:
print(y_pred[:5])

[335000. 140200. 119000. 207500. 112000.]


In [17]:
pd.DataFrame({'y' : y_valid.head(), 'y_preds' : y_pred})

,y,y_preds
Id,,
530,200624,335000.0
492,133000,140200.0
460,110000,119000.0
280,192000,207500.0
656,88000,112000.0


In [18]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [19]:
rf_value = rf_model.predict(X_valid)
rf_value[:5]

array([271690., 155039., 122024., 188915.,  91147.])

## Predict with a new input

In [20]:
X_valid.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
530,32668,1957,2515,0,3,4,9
492,9490,1941,958,620,1,3,5
460,7015,1950,979,224,1,3,5
280,10005,1977,1156,866,2,4,8
656,1680,1971,525,567,1,3,6


In [21]:
#Predict luôn nhận mảng 2 chiều
rf_model.predict([[7015,1950,979,224,1,3,5]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([122024.])